In [1]:
%load_ext autoreload
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%autoreload

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from keras.optimizers import RMSprop
from keras import backend as K
%matplotlib inline

import pickle

#
from vgg16_siamese import vgg16_siamese, contrastive_loss
import data_processing as data_p

IMG_DIR = "../../data/ssd-images/"

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
# load training data
train_batches = pickle.load(open('./batches/fast_train_batches.pkl', 'rb'))
val_batches = pickle.load(open('./batches/fake_train_batches.pkl', 'rb'))
test_batches = pickle.load(open('./batches/fast_test_batches.pkl', 'rb'))

In [10]:
input_shape = (64, 64, 3) # 64x64 rgb images
siamese_network = vgg16_siamese(input_shape)

siamese_network.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
model_5 (Model)                 (None, 512)          16026432    input_8[0][0]                    
                                                                 input_9[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 1)            0           model_5[1][0]                    
          

In [11]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

def to_img_pairs(batch):
    return data_p.df_to_img_pair(batch, IMG_DIR)

train_img_batches = map(to_img_pairs, train_batches)
val_img_batches = map(to_img_pairs, val_batches)
test_img_batches = map(to_img_pairs, test_batches)

rms = RMSprop()
siamese_network.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
siamese_network.fit_generator(
    train_img_batches,
    steps_per_epoch=500,
    epochs=4,
    validation_data=val_img_batches,
    validation_steps=500,
)

Epoch 1/4
500/500 [==============================] - 498s 996ms/step - loss: 0.2144 - accuracy: 0.7093 - val_loss: 0.1622 - val_accuracy: 0.7855
Epoch 2/4
500/500 [==============================] - 475s 950ms/step - loss: 0.1701 - accuracy: 0.7642 - val_loss: 0.1526 - val_accuracy: 0.8064
Epoch 3/4
500/500 [==============================] - 475s 950ms/step - loss: 0.1608 - accuracy: 0.7838 - val_loss: 0.1536 - val_accuracy: 0.8118
Epoch 4/4
500/500 [==============================] - 475s 950ms/step - loss: 0.1484 - accuracy: 0.8007 - val_loss: 0.1578 - val_accuracy: 0.8014


In [12]:
# compute final accuracy on training and test sets
tr_img_batches = map(to_img_pairs, train_batches)
te_img_batches = map(to_img_pairs, test_batches)


# y_pred = model.predict(tr_img_batches)
# tr_acc = compute_accuracy(tr_y, y_pred)
# y_pred = model.predict(te_img_batches)
# te_acc = compute_accuracy(te_y, y_pred)

tr_loss, tr_acc = siamese_network.evaluate_generator(tr_img_batches, steps=50)
te_loss, te_acc = siamese_network.evaluate_generator(te_img_batches, steps=100)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

* Accuracy on training set: 81.38%
* Accuracy on test set: 79.16%
